In [1]:
import os
%pwd

'/home/petar/Documents/python_projects/petar-milivojevic-mlhU-machine-learning-new-mP1l/research'

In [2]:
os.chdir('../')
%pwd

'/home/petar/Documents/python_projects/petar-milivojevic-mlhU-machine-learning-new-mP1l'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    dataset_folder: Path
    dataset_xlsx: Path
    train_dataset: Path
    test_dataset: Path
    params: dict
    dataset_val_status: bool

In [4]:
from aiusage.constant import *
from aiusage.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        dataset_val_status_file = self.config.data_validation.STATUS_FILE
        
        with open(dataset_val_status_file, 'r') as f:
            status = f.read()
        
        status = bool(str.split(status)[-1])
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            dataset_folder=config.dataset_folder,
            dataset_xlsx=config.dataset_xlsx,
            train_dataset=config.train_dataset,
            test_dataset=config.test_dataset,
            params=self.params,
            dataset_val_status=status
        )
        
        return data_transformation_config

In [7]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import json

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def format_save_data(self):
        if self.config.dataset_val_status:
            data_df = pd.read_excel(self.config.dataset_xlsx)

            data_array = []

            for _, row in data_df.iterrows():
                text = ' Question: '

                with open(os.path.join(
                        self.config.dataset_folder,
                        row['coding_problem_id'],
                        row['coding_problem_id'] + '.json'
                    ), 'r') as f:
                
                    file = json.load(f)

                text += file['question'] + ' Candidate code: '

                for dir in os.listdir(os.path.join(self.config.dataset_folder, row['coding_problem_id'])):
                    if row['coding_problem_id'] + '.' in dir and 'json' not in dir:
                        file_path = os.path.join(self.config.dataset_folder, row['coding_problem_id'], dir)

                        with open(file_path, 'r') as f:
                            script_file = f.read()

                        text += script_file + ' AI Code: '
                        break

                for dir in os.listdir(os.path.join(self.config.dataset_folder, row['coding_problem_id'])):
                    if row['llm_answer_id'] in dir:
                        file_path = os.path.join(self.config.dataset_folder, row['coding_problem_id'], dir)
                
                        with open(file_path, 'r') as f:
                            script_file = f.read()

                        text += script_file
                        break
                        
                data_array.append(text)
            
            labels_array = list(data_df['plagiarism_score'])

            train_array, test_array, train_labels_array, test_labels_array = train_test_split(
                data_array,
                labels_array,
                test_size=self.config.params.test_split,
                random_state=42
            )

            train_data = pd.DataFrame(
                {
                    'sample': train_array,
                    'label': train_labels_array
                }
            )

            train_data.to_csv(self.config.train_dataset, index=False)

            test_data = pd.DataFrame(
                {
                    'sample': test_array,
                    'label': test_labels_array
                }
            )

            test_data.to_csv(self.config.test_dataset, index=False)
        else:
            print("Dataset didn't pass validation!!!")

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.format_save_data()

except Exception as e:
    raise e

[2025-01-08 06:23:55,837: INFO: main_utils: created directory at: artifacts]
[2025-01-08 06:23:55,838: INFO: main_utils: created directory at: artifacts/data_transformation]
